In [19]:
import pandas as pd

from sqlalchemy import create_engine
import sqlite3
from sqlite3 import Error
from contextlib import contextmanager

DB_NAME = "flashcard_ai.db"


def add_questions_to_table(df):
    engine = create_engine("sqlite:///flashcard_ai.db")
    table_name = "flashcards"
    df.to_sql(table_name, engine, if_exists="append", index=False)
    return "Added questions to table"


@contextmanager
def get_db():
    # create db connection
    try:
        conn = sqlite3.connect(DB_NAME)
        yield conn
    except Error as e:
        raise
    finally:
        if conn:
            conn.close()


def select_flashcard_by_id(id):
    with get_db() as conn:
        cur = conn.cursor()
        try:
            cur.execute("SELECT * FROM flashcards WHERE rowid = ?", (id,))
            for row in cur:
                return dict(
                    (column[0], row[index])
                    for index, column in enumerate(cur.description)
                )
        except Error as e:
            raise


def select_all_flashcards():
    with get_db() as conn:
        cur = conn.cursor()
        try:
            cur.execute("SELECT * FROM flashcards")
            return cur.fetchall()
        except Error as e:
            raise

In [20]:
from fsrs import *
from datetime import datetime, UTC

import sqlite3
from sqlite3 import Error
from contextlib import contextmanager
from database import get_db
from fsrs import *


class flashcard(Card):
    def __init__(self, has_preamble, preamble_text, question, answer, *args, **kwargs):
        super().__init__()
        if has_preamble == "Yes":
            self.has_preamble = True
        else:
            self.has_preamble = False
        self.preamble_text = preamble_text
        self.question = question
        self.answer = answer

    def __repr__(self):
        if self.has_preamble:
            return f"""Card: 
                preamble: {self.preamble_text} 
                question: {self.question} 
                answer: {self.answer} 
                due: {self.due} 
                stability: {self.stability}
                difficulty: {self.difficulty}
                elapsed_days:{self.elapsed_days}
                scheduled_days: {self.scheduled_days}
                reps: {self.reps}
                laspes: {self.lapses}
                state: {self.state}
            """
        else:
            return f"""Card: 
                    question: {self.question} 
                    answer: {self.answer} 
                    due: {self.due} 
                    stability: {self.stability}
                    difficulty: {self.difficulty}
                    elapsed_days:{self.elapsed_days}
                    scheduled_days: {self.scheduled_days}
                    reps: {self.reps}
                    laspes: {self.lapses}
                    state: {self.state}
                """

    def update_card(self, id_, *args, **kwargs):
        """
        Updates a card in the database with the given ID.

        Parameters:
            id_ (int): The ID of the card to update.
            *args: Variable length argument list.
            **kwargs: Arbitrary keyword arguments.

        Returns:
            str: A message indicating that the card has been updated.

        Raises:
            Error: If there is an error executing the SQL query.

        This function updates a card in the database with the given ID. It first checks if the ID is provided and assigns it to the `id_` attribute of the class. Then, it gets a database connection and executes an SQL query to update the card in the `flashcards` table. The query updates the `due`, `stability`, `difficulty`, `elapsed_days`, `scheduled_days`, `reps`, `lapses`, and `state` columns of the card with the corresponding values from the class attributes. The card is identified by its ID. If the update is successful, the function commits the changes and returns a message indicating that the card has been updated. If there is an error executing the SQL query, it raises an `Error` exception.
        """
        self.id_ = id_
        # get database connection
        with get_db() as conn:
            cur = conn.cursor()
            try:
                cur.execute(
                    "UPDATE flashcards SET due = ?, stability = ?, difficulty = ?, elapsed_days = ?, scheduled_days = ?, reps = ?, lapses = ?, state = ? WHERE rowid = ?",
                    (
                        self.due,
                        self.stability,
                        self.difficulty,
                        self.elapsed_days,
                        self.scheduled_days,
                        self.reps,
                        self.lapses,
                        self.state,
                        self.id_,
                    ),
                )
                conn.commit()
                return "Updated card"
            except Error as e:
                raise

    def review_card(self, scheduler, *args, **kwargs):
        time = datetime.now(UTC)
        scheduling_cards = scheduler.repeat(self, time)
        return scheduling_cards

    def rate_easy(self, scheduling_cards):
        card_easy = scheduling_cards[Rating.Easy].card
        return card_easy

    def rate_good(self, scheduling_cards):
        card_good = scheduling_cards[Rating.Good].card
        return card_good

    def rate_hard(self, scheduling_cards):
        card_hard = scheduling_cards[Rating.Hard].card
        return card_hard

    def rate_again(self, scheduling_cards):
        card_again = scheduling_cards[Rating.Again].card
        return card_again

In [21]:
import pandas as pd


def load_data(file_path, sheet_name):
    card_columns = {
        "due": None,
        "stability": 0,
        "difficulty": 0,
        "elapsed_days": 0,
        "scheduled_days": 0,
        "reps": 0,
        "lapses": 0,
        "state": None,
    }
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    df = df.assign(**card_columns)
    df.rename(columns={"S/N": "Question_number"}, inplace=True)
    return df

In [22]:
df = load_data("data/2021/2021 NSMQ contest 9.xlsx", "Round 1")
df.head()

,Question_number,Has Preamble,Preamble Text,Question,Question has figure,Question figure name,Answer has figure,Answer figure name,Answer,calculations present,Subject,due,stability,difficulty,elapsed_days,scheduled_days,reps,lapses,state
0,1,Yes,Solve the equation for $x$,$\log \left(x^{2}-15 x\right)=2$,No,NaN,No,NaN,"$x=20,-5$\n$\left[x^{2}-15 x=100, x^{2}-15 x-1...",Yes,Mathematics,None,0,0,0,0,0,0,None
1,2,Yes,Solve the equation for $x$,$\log _{3}\left(x^{2}+6 x\right)=3$,No,NaN,No,NaN,"$x=3,-9$\n$\left[x^{2}+6 x=27, x^{2}+6 x-27=(x...",Yes,Mathematics,None,0,0,0,0,0,0,None
2,3,Yes,Solve the equation for $x$,"$\log _{5}\left(x^{2}+24 x\right)=2$,",No,NaN,No,NaN,"$x=-25,1$\n$\left[x^{2}+24 x=25, x^{2}+24 x-25...",Yes,Mathematics,None,0,0,0,0,0,0,None
3,4,Yes,$\mathrm{n}$ is a positive integer. Solve for ...,$\mathrm{nC}_{2}=36 \quad($ read as ' $\mathrm...,No,NaN,No,NaN,"$\mathbf{n}=\mathbf{9}$\n$\left[n(n-1) / 2=36,...",Yes,Mathematics,None,0,0,0,0,0,0,None
4,5,Yes,$\mathrm{n}$ is a positive integer. Solve for ...,$(\mathrm{n}+1) \mathrm{C}_{2}=78$,No,NaN,No,NaN,"$n=12$\n$\left[(n+1) n / 2=(78), n^{2}+n-156=(...",Yes,Mathematics,None,0,0,0,0,0,0,None


In [23]:
add_questions_to_table(df)

'Added questions to table'

In [24]:
select_all_flashcards()

[(1,
  'Yes',
  'Solve the equation for $x$',
  '$\\log \\left(x^{2}-15 x\\right)=2$',
  'No',
  None,
  'No',
  None,
  '$x=20,-5$\n$\\left[x^{2}-15 x=100, x^{2}-15 x-100=(x-20)(x+5)=0, x=20,-5\\right]$',
  'Yes',
  'Mathematics',
  None,
  0,
  0,
  0,
  0,
  0,
  0,
  None),
 (2,
  'Yes',
  'Solve the equation for $x$',
  '$\\log _{3}\\left(x^{2}+6 x\\right)=3$',
  'No',
  None,
  'No',
  None,
  '$x=3,-9$\n$\\left[x^{2}+6 x=27, x^{2}+6 x-27=(x+9)(x-3)=0, x=3,-9\\right]$',
  'Yes',
  'Mathematics',
  None,
  0,
  0,
  0,
  0,
  0,
  0,
  None),
 (3,
  'Yes',
  'Solve the equation for $x$',
  '$\\log _{5}\\left(x^{2}+24 x\\right)=2$,',
  'No',
  None,
  'No',
  None,
  '$x=-25,1$\n$\\left[x^{2}+24 x=25, x^{2}+24 x-25=(x+25)(x-1)=0, x=1,-25\\right]$',
  'Yes',
  'Mathematics',
  None,
  0,
  0,
  0,
  0,
  0,
  0,
  None),
 (4,
  'Yes',
  '$\\mathrm{n}$ is a positive integer. Solve for $\\mathrm{n}$ from the equation',
  "$\\mathrm{nC}_{2}=36 \\quad($ read as ' $\\mathrm{n}$ combinati

In [25]:
flashcard_params = [
    "Has Preamble",
    "Preamble Text",
    "Question",
    "Answer",
    "due",
    "stability",
    "difficulty",
    "elapsed_days",
    "scheduled_days",
    "reps",
    "lapses",
    "state",
]

In [38]:
current_card = select_flashcard_by_id(10)
card = flashcard(*flashcard_params)
f = FSRS()
scheduling_cards = card.review_card(f)
scheduling_cards

{<Rating.Again: 1>: <fsrs.models.SchedulingInfo at 0x12fd88990>,
 <Rating.Hard: 2>: <fsrs.models.SchedulingInfo at 0x12fd88490>,
 <Rating.Good: 3>: <fsrs.models.SchedulingInfo at 0x12fd89350>,
 <Rating.Easy: 4>: <fsrs.models.SchedulingInfo at 0x12fd88ad0>}

In [39]:
card_reviewed = card.rate_easy(scheduling_cards)
card_reviewed

Card: 
                    question: Question 
                    answer: Answer 
                    due: 2024-06-12 10:11:21.305250+00:00 
                    stability: 5.8
                    difficulty: 3.9899999999999998
                    elapsed_days:0
                    scheduled_days: 6
                    reps: 1
                    laspes: 0
                    state: 2
                

In [40]:
card_reviewed.update_card(10)

'Updated card'

In [41]:
select_flashcard_by_id(10)

{'Question_number': 10,
 'Has Preamble': 'No',
 'Preamble Text': None,
 'Question': 'The human body is composed of just four basic kinds of tissue. Name one each.',
 'Question has figure': 'No',
 'Question figure name': None,
 'Answer has figure': 'No',
 'Answer figure name': None,
 'Answer': 'Nervous\nMuscular\nEpithelial\nConnective tissue',
 'calculations present': 'No',
 'Subject': 'Biology',
 'due': '2024-06-12 10:11:21.305250+00:00',
 'stability': 5.8,
 'difficulty': 3.9899999999999998,
 'elapsed_days': 0,
 'scheduled_days': 6,
 'reps': 1,
 'lapses': 0,
 'state': '2'}